In [72]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [86]:
airlines_data = pd.read_csv("data/airlines.csv")
airlines_data = airlines_data[["src_iata", "src_id", "dest_iata", "dest_id"]]
airlines_data

KeyError: 'True: boolean label can not be used without a boolean index'

In [66]:
airports_data = pd.read_csv("data/airports.csv")
airports_data = airports_data[["id", "iata", "latitude", "longitude", "altitude"]]
airports_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7698 entries, 0 to 7697
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         7698 non-null   int64  
 1   iata       7698 non-null   object 
 2   latitude   7698 non-null   float64
 3   longitude  7698 non-null   float64
 4   altitude   7698 non-null   int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 300.8+ KB


In [62]:
merged_src = pd.merge(airlines_data, airports_data, left_on='src_id', right_on='id', how='left', suffixes=('_src','_airports_src'))


ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [61]:
G = nx.DiGraph()

for _, row in data.iterrows():
    G.add_edge(row['src_name'], row['dest_name'], src_id=row['src_id'], dest_id=row['dest_id'])

nx.draw(G, with_labels=True)
plt.show()

KeyboardInterrupt: 